# RePsychLing &#39;Masson, Rabe, &amp; Kliegl, 2017)&#39; with Julia
### Reinhold Kliegl
### 2020-02-13
# Update

This version uses `MixedModels.PCA()` to show details about RE structures. 

# Setup

Packages we (might) use.

In [ ]:
#cd(joinpath(homedir(),"Google Drive/ZiF_CG_WS2/MRK17_Exp1/"))
#cd("/Users/reinholdkliegl/Google Drive/ZiF_CG_WS2/MRK17_Exp1/")

using CSV, DataFrames, DataFramesMeta, MixedModels, RCall 
using StatsBase, StatsModels, BenchmarkTools

# earlier alternative for for LRT
# using Printf: @sprintf
# using Distributions: Chisq, ccdf
# include("lrtest.jl")

# Reading data

We read the data preprocessed with R and saved as RDS file (see `DataPrep.Rmd` for details).

In [ ]:
R"dat_r = readRDS('MRK17_Exp1.rds')";

dat = rcopy(R"dat_r")

dat = @linq dat |>
       transform(F = levels!(:F, ["HF", "LF"]),
                 P = levels!(:P, ["rel", "unr"]),
                 Q = levels!(:Q, ["clr", "deg"]),
                lQ = levels!(:lQ, ["clr", "deg"]),
                lT = levels!(:lT, ["WD", "NW"]))

cellmeans = by(dat, [:F, :P, :Q, :lQ, :lT], 
            meanRT = :rt => mean, sdRT = :rt => std, n = :rt => length,
            semean = :rt => x -> std(x)/sqrt(length(x)))

# Complex LMM

This is *not* the maximal factorial LMM because we do not include interaction 
terms and associated correlation parameters in the RE structure.

## Model fit

In [ ]:
const HC = HelmertCoding();
const contrasts = Dict(:F => HC, :P => HC, :Q => HC, :lQ => HC, :lT => HC);

m1form = @formula (-1000/rt) ~ 1+F*P*Q*lQ*lT +
                              (1+F+P+Q+lQ+lT | Subj) +
                              (1+P+Q+lQ+lT | Item);
cmplxLMM = @btime fit(MixedModel, m1form, dat, contrasts=contrasts);

## VCs and CPs

We don't look at fixed effects before model selection.

In [ ]:
cmplxLMM.λ[1]

In [ ]:
cmplxLMM.λ[2]

##  rePCA

Options for information about rePCAs

```
Base.show(pca::PCA;
          ndigitsmat=2, ndigitsvec=2, ndigitscum=4,
          covcor=true, loadings=true, variances=false, stddevs=false)
```

In [ ]:
cmplxLMM.rePCA

In [ ]:
cmplx_pca=MixedModels.PCA(cmplxLMM, corr=false);
show(stdout, cmplx_pca.Subj, ndigitsmat=4, ndigitsvec=6, variances=true, stddevs=true)
show(stdout, cmplx_pca.Item, ndigitsmat=4, ndigitsvec=6, variances=true, stddevs=true)

In [ ]:
cmplx_pca=MixedModels.PCA(cmplxLMM, corr=true);
show(stdout, cmplx_pca.Subj)
show(stdout, cmplx_pca.Item)

In [ ]:
cmplx_pca=MixedModels.PCA(cmplxLMM, corr=false);
show(stdout, cmplx_pca.Subj)
show(stdout, cmplx_pca.Item, stddevs=true, loadings=false)

Variance-covariance matrix of random-effect structure suggests overparameterization
for both subject-related and item-related components.


# Zero-correlation parameter LMM (factors)

## Model fit

We take out correlation parameters.

In [ ]:
m2form = @formula (-1000/rt) ~ 1 + F*P*Q*lQ*lT +
                               zerocorr(1+F+P+Q+lQ+lT | Subj) +
                               zerocorr(1+P+Q+lQ+lT | Item);

zcpLMM = @btime fit(LinearMixedModel, m2form, dat, contrasts=contrasts);

## VCs and CPs

In [ ]:
zcpLMM.λ[1]

In [ ]:
zcpLMM.λ[2]

##  rePCA

Options for information about rePCAs

```
Base.show(pca::PCA;
          ndigitsmat=2, ndigitsvec=2, ndigitscum=4,
          covcor=true, loadings=true, variances=false, stddevs=false)
```

In [ ]:
show(stdout, zcpLMM.rePCA)

zcp_pca=MixedModels.PCA(zcpLMM, corr=true);
show(stdout, zcp_pca.Subj, stddevs=true)
show(stdout, zcp_pca.Item, stddevs=true)

Looks ok, but the last PCs are very tiny. Might be a good idea to prune the LMM. 

# Zero-correlation parameter LMM (indicators)

An alternative solution is to extract the indicators of contrasts from the design matrix.
Sometimes RE structures are more conviently specified with indicator variables (i.e., 
@ level of contrasts) than the factors.

In [ ]:
mm = Int.(zcpLMM.X)

dat = @linq dat |>
       transform(f = mm[:, 2],
                 p = mm[:, 3],
                 q = mm[:, 4],
                lq = mm[:, 5],
                lt = mm[:, 6]);
dat[1:10, 10:14]

We take out correlation parameters.

In [ ]:
m2form_b = @formula (-1000/rt) ~ 1 + f*p*q*lq*lt +
 (1 | Subj) + (0+f | Subj) + (0+p | Subj) + (0+q | Subj) + (0+lq | Subj) + (0+lt | Subj) +
 (1 | Item) +                (0+p | Item) + (0+q | Item) + (0+lq | Item) + (0+lt | Item);

zcpLMM_b = @btime fit(LinearMixedModel, m2form_b, dat, contrasts=contrasts);

const mods = [cmplxLMM, zcpLMM, zcpLMM_b];

In [ ]:
gof_summary = DataFrame(dof=dof.(mods), deviance=deviance.(mods),
              AIC = aic.(mods), AICc = aicc.(mods), BIC = bic.(mods))

In [ ]:
#lrtest(cmplxLMM, zcpLMM) # earlier alternative
MixedModels.likelihoodratiotest(zcpLMM, cmplxLMM)

Results are identical; goodness of fit is better for complex LMM -- 
marginally because 2 * ΔDOF < ΔDeviance). 

# A replication of MRK17 LMM

## Indicators

Replication of final LMM in Masson and Kliegl (2013, Table 1) as well as
reproduction of final lme4-based LMM in Masson, Rabe, and Kliegl (2017, Figure 2)

In [ ]:
m3form = @formula (-1000/rt) ~ 1 + f*p*q*lq*lt +
        (1+q | Subj) + (0+lt | Subj) + (1 | Item) + (0 + p | Item) ;
mrk17_LMM = @btime fit(LinearMixedModel, m3form, dat, contrasts=contrasts);

VarCorr(mrk17_LMM)

Is the correlation paramter significant?

In [ ]:
# remove single CP for nested LMMs
m4form = @formula (-1000/rt) ~ 1 + f*p*q*lq*lt +
        (1 | Subj) + (0+q | Subj) + (0+lt | Subj) + (1 | Item) + (0+p | Item);
rdcdLMM = @btime fit(LinearMixedModel, m4form, dat, contrasts=contrasts);

#compare nested model sequence
# lrtest(rdcdLMM, mrk17_LMM)  # earlier alternative
MixedModels.likelihoodratiotest(rdcdLMM, mrk17_LMM)

Yes, it is! Replicates a previous result. 

Note that `zcpLMM` and `mrk17LMM` are not nested; we cannot compare them with a LRT.

## rePCA

Options for information about rePCAs
 
```
Base.show(pca::PCA;
          ndigitsmat=2, ndigitsvec=2, ndigitscum=4,
          covcor=true, loadings=true, variances=false, stddevs=false)
```

In [ ]:
mrk17_LMM.rePCA

In [ ]:
mrk17_pca=MixedModels.PCA(mrk17_LMM, corr=true);
show(stdout, mrk17_pca.Subj, stddevs=true)
show(stdout, mrk17_pca.Item, stddevs=true)

VarCorr(mrk17_LMM)

## Factors

This is an excursion with cautionary note. 
The replication LMM cannot be specified with factors in the RE-structure.

In [ ]:
m3form_b = @formula (-1000/rt) ~ 1 + F*P*Q*lQ*lT +
        (1+Q | Subj) + zerocorr(0+lT | Subj) + zerocorr(1 + P | Item) ;
mrk17_LMM_b = @btime fit(LinearMixedModel, m3form_b, dat, contrasts=contrasts);

VarCorr(mrk17_LMM_b)

In [ ]:
VarCorr(mrk17_LMM)

This will be fixed (see https://github.com/JuliaStats/MixedModels.jl/issues/268)

# Model comparisons

In [ ]:
const mods = [cmplxLMM, zcpLMM, mrk17_LMM, rdcdLMM];
gof_summary = DataFrame(dof=dof.(mods), deviance=deviance.(mods),
              AIC = aic.(mods), AICc = aicc.(mods), BIC = bic.(mods))

Here `dof` or degrees of freedom is the total number of parameters estimated 
in the model and `deviance` is simply negative twice the log-likelihood at 
convergence, without a correction for a saturated model.  The There information 
criteria are on a scale of "smaller is better" and all would select `mrk17_LMM` as "best".

The correlation parameter was replicated (i.e., -.42 in MRK17)

# Illustration of crossing and nesting of factors

There is an implementation of Wilkinson & Rogers (1973) formula syntax, allowing the specification of factors not only as crossed, but also as nested in the levels of another factor or combination of factors. We illustrate this functionality with a subset of the MRK17 data. (We use oviLMM as RE structure and rt as dependent variable.)

## Crossing factors

The default analysis focuses on crossed factors yielding main effects and interactions.

In [ ]:
m5form = @formula rt ~ 1 + F*P + (1 | Subj) + (1 | Item);
crossedLMM = @btime fit(LinearMixedModel, m5form, dat, contrasts=contrasts)

Main effects of frequency (F) and priming (P) and their interaction are significant.

In [ ]:
cellmeans = by(dat, [:F, :P], 
            meanRT = :rt => mean, sdRT = :rt => std, n = :rt => length)

# Nesting factors

The interaction tests whether lines visualizing the interaction are parallel, 
but depending on the theoretical context one might be interested whether the
priming effect is significant high frequency targets and for low frequency targets. 
In other words, the focus is on whether the priming effect is significant for 
the levels of the frequency factor.

In [ ]:
m6form = @formula rt ~ 1 + F/P + (1 | Subj) + (1 | Item);
nestedLMM = @btime fit(LinearMixedModel, m6form, dat, contrasts=contrasts)

The results show that the priming effect is not significant for high-frequency
targets. The estimates are the differences of the cell means from the grand 
mean (i.e., 2 x estimate = effect).